# 정치

In [133]:
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd

# 정치면 메인 페이지 요청 -> html로 파싱
url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=100'
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
req = requests.get(url, headers = headers)
target = bs(req.content, 'html.parser')

url_main = 'https://news.naver.com/'

total = {}
link = {}
for idx, tag in enumerate(target.select('div.cluster_foot_inner > a')):
    
    each_url = url_main + tag['href'] # 각 헤드라인 서브 페이지 링크
    
    try: 
        news_num = int(re.match('[0-9]+', tag.text).group()) # 관련 기사 갯수
    except:
        news_num = np.nan
        
    # 헤드라인 서브 페이지 요청 -> html로 파싱
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
    sub_req = requests.get(each_url, headers = headers)
    page = bs(sub_req.content, 'html.parser')
    
    # 헤드라인 서브 페이지에서 가장 첫번째 기사 제목 (편의상 넣음 꼭 필요한 것은 아님)
    topic = page.select('div > ul > li > dl > dt')[1].text.strip() 
    
    # link 딕셔너리에 헤드라인 서브 페이지 링크, 관련 기사 건수, 첫번째 기사 제목  
    link[idx] = {'head_link':each_url, '관련기사':news_num, 'topic': topic} 
    
    # 헤드라인 서브 페이지 별로 언론사 리스트 추출
    np_list = page.select('span.writing')
    counter = {}
    for np_name in np_list:
        if np_name.text in ['조선일보', '동아일보', '경향신문', '한겨레', '한국일보', '중앙일보']:
            if np_name.text not in counter:
                counter[np_name.text] = 0
            counter[np_name.text] += 1
        else:
            pass
    total[idx] = counter # key: index / value : 언론사 별 기사 

df1 = pd.DataFrame.from_dict(link, orient = 'index') # 링크, 관련기사 수, topic 있는 dataframe
df2 = pd.DataFrame.from_dict(total, orient = 'index') # 각 언론사별 기사 갯수 있는 dataframe

# merge
df3 = pd.merge(df1.reset_index(), df2.reset_index(), on = 'index', how = 'outer').drop('index', axis = 1)

con = ['조선일보', '동아일보']
pro = ['경향신문', '한겨레']

df3['보수'] = df3[con].notna().sum(axis = 1) # 보수신문 2개 있는지
df3['진보'] = df3[pro].notna().sum(axis = 1) # 진보신문 2개 있는지
df3['합계'] = df3['보수'] + df3['진보']
df3

,head_link,관련기사,topic,조선일보,중앙일보,한겨레,경향신문,동아일보,한국일보,보수,진보,합계
0,https://news.naver.com//main/clusterArticles.n...,10,,1.0,NaN,NaN,NaN,NaN,NaN,1,0,1
1,https://news.naver.com//main/clusterArticles.n...,49,"송영길 ""尹 돌잔치 엔화 발언 실수인정""…野 ""급하긴 급한 모양""",2.0,2.0,1.0,1.0,1.0,1.0,2,2,4
2,https://news.naver.com//main/clusterArticles.n...,18,"민주당, 이재명 뜻 받아들여 선대위 전면쇄신…""민주당도 확 바꾸자""",NaN,NaN,NaN,1.0,NaN,NaN,0,1,1
3,https://news.naver.com//main/clusterArticles.n...,18,"홍준표 뒤끝? '尹 전화 왜 안받나' 질문에 ""받아본들 할말 없다""",NaN,1.0,NaN,NaN,NaN,1.0,0,0,0
4,https://news.naver.com//main/clusterArticles.n...,40,"김두관 ""공동선대위원장 사퇴하겠다""",1.0,2.0,NaN,1.0,NaN,2.0,1,1,2
5,https://news.naver.com//main/clusterArticles.n...,12,"이재명, 선대위 쇄신 예고…윤석열·김종인, 김병준·김한길 영입 합의",NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
6,https://news.naver.com//main/clusterArticles.n...,41,문대통령 내일 '국민과의 대화'…무슨 이야기 나눌까,NaN,1.0,NaN,NaN,NaN,NaN,0,0,0
7,https://news.naver.com//main/clusterArticles.n...,34,"""베프 결혼식도 못가"" 서글픈 30대 백수...이재명·윤석열의 해법은?",1.0,1.0,NaN,1.0,1.0,1.0,2,1,3
8,https://news.naver.com//main/clusterArticles.n...,8,국힘 `배우자포럼` 준비 중…尹후보 부인 김건희 등판 플랫폼 되나,1.0,NaN,NaN,1.0,NaN,NaN,1,1,2
9,https://news.naver.com//main/clusterArticles.n...,18,"송영길, 선대위 쇄신론에 ""모든 것 비우고 새출발하자""",1.0,1.0,NaN,NaN,NaN,NaN,1,0,1


In [140]:
# 보수, 진보 2개씩 있는 df
df4 = df3[(df3['합계'] == 4)].reset_index().drop('index', axis = 1)

# 보수, 진보 합쳐서 3개 있는 df 
df4_1 = df3[(df3['합계'] == 3)].reset_index().drop('index', axis = 1)

# 보수, 진보 각각 1개 있는 df
df4_2 = df3[(df3['보수'] == 1) & df3['진보'] == 1].reset_index().drop('index', axis = 1)

if len(df4) > 0:
# 보수, 진보 2씩 있는 df - 관련기사 가장 많은 헤드라인의 링크
    max_idx = df4['관련기사'].argmax()
    selected_url = df4['head_link'][max_idx]
    print('관련기사 건수', df4['관련기사'].max())
    print('기사 4개, 선택된 url', selected_url)
    
    # 기사 4개 일때 언론사는 원래 그대로 
    new_con = con
    new_pro = pro
    print('선택한 언론사', new_pro + new_con)
    
# 보수, 진보 합쳐서 3개 있는 df - 관련기사 가장 많은 헤드라인의 링크
elif len(df4_1) > 0:
    max_idx = df4_1['관련기사'].argmax()
    selected_url = df4_1['head_link'][max_idx]
    print('관련기사 건수', df4_1['관련기사'].max())
    print('기사 3개, 선택된 url', selected_url)
    # 여기에 포함 안된 언론사
    absent_name = df4_1.loc[max_idx][pro+con].notna().idxmin()
    print('제외된 언론사', absent_name)
    
    # 기사 3개 일때 선택할 언론사
    if absent_name in pro:
        new_pro = list(set(pro) - {absent_name}) + ['한국일보']
        new_con = con
    else:
        new_pro = pro
        new_con = list(set(con) - {absent_name}) + ['중앙일보']
    print('선택한 언론사', new_pro + new_con)
    
# 보수, 진보 각각 1개 있는 df - 관련기사 가장 많은 헤드라인의 링크
elif len(df4_2) > 0:
    max_idx = df4_2['관련기사'].argmax()
    selected_url = df4_2['head_link'][max_idx]
    print('관련기사 건수', df4_2['관련기사'].max())
    print('기사 2개, 선택된 url', selected_url)
    # 여기에 포함 안된 언론사
    absent_name_con = df4_2.loc[max_idx][con].notna().idxmin()
    absent_name_pro = df4_2.loc[max_idx][pro].notna().idxmin()
    print('제외된 언론사', absent_name_con, absent_name_pro)
    
    # 보수, 진보 각각 1개씩 있을 때 선택할 언론사
    new_pro = list(set(pro) - {absent_name_pro}) + ['한국일보']
    new_con = list(set(con) - {absent_name_con}) + ['중앙일보']
    print('선택한 언론사', new_pro + new_con)

관련기사 건수 49
기사 4개, 선택된 url https://news.naver.com//main/clusterArticles.naver?id=c_202111192040_00000149&mode=LSD&mid=shm&sid1=100&oid=025&aid=0003152345
선택한 언론사 ['경향신문', '한겨레', '조선일보', '동아일보']


In [141]:
# 선택된 헤드라인 페이지에서 신문사별로 링크 따오기
def get_links(selected_url):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
    req = requests.get(selected_url, headers = headers)
    target = bs(req.content, 'html.parser')

    con_link1 = {} # 보수 언론사 1
    con_link2 = {} # 보수 언론사 2
    pro_link1 = {} # 진보 언론사 1
    pro_link2 = {} # 진보 언론사 2
    con_num1 = 0 
    con_num2 = 0
    pro_num1 = 0
    pro_num2 = 0

    for tag in target.select('div > ul > li > dl'):
        if tag.select_one('dd > span.writing').text == new_con[0]: # 보수 언론사 1 이름과 동일할 경우
            con_num1 += 1 # 기사 번호
            con_link1[con_num1] = tag.select_one('dt > a')['href'] # 기사 번호 당 링크
        if tag.select_one('dd > span.writing').text == new_con[1]: # 보수 언론사 2 이름과 동일할 경우
            con_num2 += 1 # 기사 번호
            con_link2[con_num2] = tag.select_one('dt > a')['href'] # 기사 번호 당 링크
        if tag.select_one('dd > span.writing').text == new_pro[0]: # 진보 언론사 1 이름과 동일할 경우
            pro_num1 += 1 # 기사 번호
            pro_link1[pro_num1] = tag.select_one('dt > a')['href'] # 기사 번호 당 링크
        if tag.select_one('dd > span.writing').text == new_pro[1]: # 진보 언론사 2 이름과 동일할 경우
            pro_num2 += 1 # 기사 번호
            pro_link2[pro_num2] = tag.select_one('dt > a')['href'] # 기사 번호 당 링크
        else:
            pass

    links = {new_con[0]: con_link1, new_con[1]: con_link2, new_pro[0]: pro_link1, new_pro[1]: pro_link2}
    return links
total_links = get_links(selected_url)
print(total_links)

{'조선일보': {1: 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=100&oid=023&aid=0003654361', 2: 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=100&oid=023&aid=0003654404'}, '동아일보': {1: 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=100&oid=020&aid=0003394916'}, '경향신문': {1: 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=100&oid=032&aid=0003111370'}, '한겨레': {1: 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=100&oid=028&aid=0002568562'}}


In [142]:
# 동일한 언론사 내에서 기사가 2건 이상 있을 경우 
# 해당 언론사의 여러 기사 페이지에서 시간 추출하여 서로 비교하고 최신 기사를 추출
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime

def compare_time(links):
    article_times = {} 
    for key, url in links.items():
        headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
        req = requests.get(url, headers = headers)
        target = bs(req.content, 'html.parser')
        time = target.select('span.t11')[-1].text # 기사 입력 시간 추출 (최초 작성 후 수정본이 있을때 수정된 시간으로 추출 / 만약 최초 작성 시간 기준을 하고 싶으면 인덱스[0]) 
        time = time.replace('오후', 'PM').replace('오전', 'AM') # 오후 -> PM, 오전 -> AM 변경
        time = datetime.strptime(time, '%Y.%m.%d. %p %I:%M') # datetime 으로 파싱
        article_times[key] = time # key: 원래 딕셔너리의 key 입력, value: 기사 입력 시간
    selected_article = links[max(article_times, key = article_times.get)] # article_times 딕셔너리의 value가 max인(가장 최근인) key로 con_link1 딕셔너리의 value(링크)찾기
    return selected_article

final_links = {}
for name, link in total_links.items():
    final_links[name] = compare_time(link)

print(final_links)

{'조선일보': 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=100&oid=023&aid=0003654404', '동아일보': 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=100&oid=020&aid=0003394916', '경향신문': 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=100&oid=032&aid=0003111370', '한겨레': 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=100&oid=028&aid=0002568562'}


In [143]:
# 정치면 기사 dataframe

def get_dataframe(final_links):
    time = []
    media = []
    head = []
    body = []

    for name, link in final_links.items():
        headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
        req = requests.get(link, headers=headers)
        soup = bs(req.content, 'html.parser')
        time.append(soup.select_one('span.t11').text) # time
        media.append(soup.select_one('div.article_header > div.press_logo > a > img')['title']) # media
        title = soup.find(id= 'articleTitle')
        try:
            head.append(title.text)
            text = soup.find(id = 'articleBodyContents')
            body.append(text.text.split('_flash_removeCallback() {}\n\n')[-1])
        except:
            continue

    df = pd.DataFrame({'time':time,'media':media,'title':head,'document':body})
    return df

get_dataframe(final_links)

,time,media,title,document
0,2021.11.20. 오후 4:22,조선일보,"송영길, ‘尹 돌상 ‘엔화’ 놓였다’ 발언에 “실수 인정한다”",\t\n\t\t\t\t\t\t\t\t더불어민주당 송영길 대표가 20일 국민의힘 윤석...
1,2021.11.20. 오후 2:32,동아일보,"송영길 “尹 돌상 ‘엔화’ 발언 실수 인정, 유감”",尹측 “엔화 아닌 한국은행 천환권…전형적 흑색선전”송영길 더불어민주당 대표가 20일...
2,2021.11.20. 오후 1:21,경향신문,"송영길, 윤석열 ‘돌상 엔화’ 발언 실수 인정…유감 표명",\t\n\t[경향신문] 송영길 더불어민주당 대표 겸 상임선거대책위원장이 20일 윤석...
3,2021.11.20. 오후 12:09,한겨레,"윤석열 쪽 “송영길 ‘돌상 엔화’ 허위사실”…송영길 “실수 인정, 유감”",윤석열 국민의힘 대선 후보가 과거 자신의 SNS에 공개한 돌잔치 사진. 20일 윤...


# 경제/사회 - 코드 동일
- 경제면 헤드라인 메인 url: https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=101
- 사회면 헤드라인 메인 url: https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=102

In [165]:
# 사회면 기준으로 작성
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import re

url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=102'
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
req = requests.get(url, headers = headers)
target = bs(req.content, 'html.parser')

url_main = 'https://news.naver.com/'

total = {}
link = {}
for idx, tag in enumerate(target.select('div.cluster_head_inner > a')):
    each_url = url_main + tag['href'] # 각 헤드라인 서브 페이지 링크
    try: 
        news_num = int(re.match('[0-9]+', tag.text).group()) # 관련 기사 갯수
    except:
        news_num = np.nan
    
    # 헤드라인 서브 페이지 요청 -> html로 파싱
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
    sub_req = requests.get(each_url, headers = headers)
    page = bs(sub_req.content, 'html.parser')
    
    # 헤드라인 서브 페이지에서 가장 첫번째 기사 제목 (편의상 넣음 꼭 필요한 것은 아님)
    topic = page.select('div > ul > li > dl > dt')[1].text.strip() # 헤드라인 별 페이지에서 가장 첫번째 기사 제목 
    
    # link 딕셔너리에 헤드라인 서브 페이지 링크, 관련 기사 건수, 첫번째 기사 제목  
    link[idx] = {'head_link':each_url, '관련기사':news_num, 'topic': topic} 
    
    # 헤드라인 서브 페이지 별로 언론사 리스트 추출
    np_list = page.select('span.writing')

    counter = {}
    for np_name in np_list:
        if np_name.text in ['조선일보', '동아일보', '경향신문', '한겨레', '한국일보', '중앙일보']:
            if np_name.text not in counter:
                counter[np_name.text] = 0
            counter[np_name.text] += 1
        else:
            pass
    total[idx] = counter # key: index / value : 언론사 별 기사 

df1 = pd.DataFrame.from_dict(link, orient = 'index') # 링크, 관련기사 수, topic 있는 dataframe
df2 = pd.DataFrame.from_dict(total, orient = 'index') # 각 언론사별 기사 갯수 있는 dataframe

# merge
df3 = pd.merge(df1.reset_index(), df2.reset_index(), on = 'index', how = 'outer').drop('index', axis = 1)

con = ['조선일보', '동아일보']
pro = ['경향신문', '한겨레']

df3['보수'] = df3[con].notna().sum(axis = 1) # 보수신문 2개 있는지
df3['진보'] = df3[pro].notna().sum(axis = 1) # 진보신문 2개 있는지
df3['합계'] = df3['보수'] + df3['진보']
df3

,head_link,관련기사,topic,중앙일보,조선일보,한국일보,한겨레,경향신문,동아일보,보수,진보,합계
0,https://news.naver.com//main/clusterArticles.n...,48.0,"""피해자 버리고 도망간 경찰 파면 요구"" 靑청원 등장",2.0,3.0,NaN,NaN,NaN,NaN,1,0,1
1,https://news.naver.com//main/clusterArticles.n...,4.0,"울진군, 수소 에너지 주민 수용성 확보 나섰다",NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
2,https://news.naver.com//main/clusterArticles.n...,9.0,"공수처, '고발사주 의혹' 조만간 결론…""윤석열 무혐의"" 관측",NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
3,https://news.naver.com//main/clusterArticles.n...,9.0,"""경찰 근무조, 남2·여1로 바뀌었다"" 익명 게시판 '여경 저격' 시끌",NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
4,https://news.naver.com//main/clusterArticles.n...,11.0,흉기 난동 현장서 이탈한 여경 “트라우마 생겨 기억 나지 않는다”,1.0,NaN,NaN,NaN,NaN,NaN,0,0,0
5,https://news.naver.com//main/clusterArticles.n...,10.0,“연인과 스님 관계 의심”...불법촬영하고 폭력 휘두른 60대 ‘집유’,NaN,1.0,NaN,NaN,NaN,NaN,1,0,1
6,https://news.naver.com//main/clusterArticles.n...,56.0,수도권·충청 공기질 '매우 나쁨'…올가을 첫 비상저감,1.0,NaN,2.0,1.0,2.0,1.0,1,2,3
7,https://news.naver.com//main/clusterArticles.n...,13.0,백신 접종완료자 78.8%...18세 이상 완료율 90.8%,NaN,NaN,NaN,NaN,1.0,NaN,0,1,1
8,https://news.naver.com//main/clusterArticles.n...,82.0,‘스토킹 신변보호’ 전 여자친구 살해한 남성 검거…“죄송하다”,3.0,2.0,2.0,1.0,2.0,1.0,2,2,4
9,https://news.naver.com//main/clusterArticles.n...,22.0,확진자 닷새째 3000명대인데…내일부터 전국 전면등교,NaN,NaN,NaN,1.0,NaN,NaN,0,1,1


In [147]:
# 보수, 진보 2개씩 있는 df
df4 = df3[(df3['합계'] == 4)].reset_index().drop('index', axis = 1)

# 보수, 진보 합쳐서 3개 있는 df 
df4_1 = df3[(df3['합계'] == 3)].reset_index().drop('index', axis = 1)

# 보수, 진보 각각 1개 있는 df
df4_2 = df3[(df3['보수'] == 1) & df3['진보'] == 1].reset_index().drop('index', axis = 1)

if len(df4) > 0:
# 보수, 진보 2씩 있는 df - 관련기사 가장 많은 헤드라인의 링크
    max_idx = df4['관련기사'].argmax()
    selected_url = df4['head_link'][max_idx]
    print('관련기사 건수', df4['관련기사'].max())
    print('기사 4개, 선택된 url', selected_url)
    
    # 기사 4개 일때 언론사는 원래 그대로 
    new_con = con
    new_pro = pro
    print('선택한 언론사', new_pro + new_con)
    
# 보수, 진보 합쳐서 3개 있는 df - 관련기사 가장 많은 헤드라인의 링크
elif len(df4_1) > 0:
    max_idx = df4_1['관련기사'].argmax()
    selected_url = df4_1['head_link'][max_idx]
    print('관련기사 건수', df4_1['관련기사'].max())
    print('기사 3개, 선택된 url', selected_url)
    # 여기에 포함 안된 언론사
    absent_name = df4_1.loc[max_idx][pro+con].notna().idxmin()
    print('제외된 언론사', absent_name)
    
    # 기사 3개 일때 선택할 언론사
    if absent_name in pro:
        new_pro = list(set(pro) - {absent_name}) + ['한국일보']
        new_con = con
    else:
        new_pro = pro
        new_con = list(set(con) - {absent_name}) + ['중앙일보']
    print('선택한 언론사', new_pro + new_con)
    
# 보수, 진보 각각 1개 있는 df - 관련기사 가장 많은 헤드라인의 링크
elif len(df4_2) > 0:
    max_idx = df4_2['관련기사'].argmax()
    selected_url = df4_2['head_link'][max_idx]
    print('관련기사 건수', df4_2['관련기사'].max())
    print('기사 2개, 선택된 url', selected_url)
    # 여기에 포함 안된 언론사
    absent_name_con = df4_2.loc[max_idx][con].notna().idxmin()
    absent_name_pro = df4_2.loc[max_idx][pro].notna().idxmin()
    print('제외된 언론사', absent_name_con, absent_name_pro)
    
    # 보수, 진보 각각 1개씩 있을 때 선택할 언론사
    new_pro = list(set(pro) - {absent_name_pro}) + ['한국일보']
    new_con = list(set(con) - {absent_name_con}) + ['중앙일보']

    print('선택한 언론사', new_pro + new_con)

관련기사 건수 82.0
기사 4개, 선택된 url https://news.naver.com//main/clusterArticles.naver?id=c_202111191720_00000052&mode=LSD&mid=shm&sid1=102&oid=052&aid=0001667581
선택한 언론사 ['경향신문', '한겨레', '조선일보', '동아일보']


In [151]:
# 선택된 헤드라인 페이지에서 신문사별로 링크 따오기 (사용자 함수 이용)
total_links = get_links(selected_url)
print(total_links)

{'조선일보': {1: 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid=023&aid=0003654393', 2: 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid=023&aid=0003654351'}, '동아일보': {1: 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid=020&aid=0003394918'}, '경향신문': {1: 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid=032&aid=0003111371', 2: 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid=032&aid=0003111373'}, '한겨레': {1: 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid=028&aid=0002568565'}}


In [152]:
# 동일한 언론사 내에서 기사가 2건 이상 있을 경우 
# 해당 언론사의 여러 기사 페이지에서 시간 추출하여 서로 비교하고 최신 기사를 추출 (사용자 함수 이용)
final_links = {}
for name, link in total_links.items():
    final_links[name] = compare_time(link)

print(final_links)

{'조선일보': 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid=023&aid=0003654393', '동아일보': 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid=020&aid=0003394918', '경향신문': 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid=032&aid=0003111371', '한겨레': 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid=028&aid=0002568565'}


In [153]:
# 사회면 기사 dataframe (사용자 함수 이용)
get_dataframe(final_links)

,time,media,title,document
0,2021.11.20. 오후 2:09,조선일보,"경찰이 신변보호 중이던 전 여자친구 살해한 30대, 대구서 검거",경찰 로고. /조선DB\t\t\t\t\t\t\t\t\t\t데이트 폭력을 당해 신변보...
1,2021.11.20. 오후 4:14,동아일보,‘신변보호 워치’ 오차…경찰 헤매는 사이 신고여성 피살,신변보호를 받던 30대 여성이 서울 시내 오피스텔에서 흉기에 찔려 사망했다. 피해자...
2,2021.11.20. 오후 2:11,경향신문,"스토킹 살해 피의자, 대구 숙박업소서 체포",\t\n\t[경향신문] 데이트폭력 피해로 경찰의 신변보호를 받던 여성을 살해하고 도...
3,2021.11.20. 오후 1:39,한겨레,신변보호받던 전 애인 찾아가 살해한 30대 남성 검거,경찰은 출동 당일 첫 신고 때 엉뚱한 곳으로 출동게티이미지 경찰의 신변보호를 받던...
